In [1]:
import data_loader as dl
import market_data_loader as mdl
import numpy as np

# ==========================================
# Parse Excel Data File
# ==========================================

file_path = r'data\data.xlsx'
invest = 90000

asset_data = dl.convert_to_csv(file_path, "assets")

alloc_target_data = dl.convert_to_csv(file_path, "allocation")
alloc_target_data = alloc_target_data.dropna(subset=['Country'])

market_data = mdl.load_market_data(asset_data)

price_data = market_data[0]
fx_data = market_data[1]
fx_data = fx_data.reindex(price_data.index).ffill()

prices_today = price_data.iloc[-1]
fx_today = fx_data.iloc[-1]

holdings_today_vec = asset_data['Current_Holdings'].to_numpy()
alloc_target_vec = alloc_target_data['Target_Allocation'].to_numpy()


# ==========================================
# Create Geogr. Exposure Matrix for Today
# ==========================================

columns_to_keep = [
    'Switzerland', 
    'Europe_ex_CH', 
    'USA', 
    'Other_Developed', 
    'Non_Developed'
]

# Filter the asset_data dataframe and convert the resulting geogr. exposure dataframe into matrix
exposure_matrix = asset_data[columns_to_keep].to_numpy(dtype=float).T


# ==========================================
# Convert Currencies to CHF
# ==========================================

# Get the currency for each asset from the original data
asset_currencies = asset_data.set_index('Ticker')['Currency']

# Ensure order of currencies is the same as prices order
aligned_currencies = asset_currencies.loc[prices_today.index]

# Reconstruct the FX ticker strings (e.g., 'USD' + 'CHF=X' -> 'USDCHF=X')
fx_keys = aligned_currencies + "CHF=X"

# Map the actual exchange rates from the fx_today series
mapped_rates = fx_keys.map(fx_today)

# Get todays asset prices in CHF
prices_chf = prices_today * mapped_rates
prices_chf_vec = prices_chf.to_numpy()

[*********************100%***********************]  7 of 7 completed
[*********************100%***********************]  2 of 2 completed


In [2]:
# Find value for each asset held in CHF today
asset_value_today_vec = holdings_today_vec * prices_chf_vec

# Find the total portfolio value today
value_total_today = np.sum(asset_value_today_vec)

# Calculate portfolio asset allocation in %
asset_weights_today_vec = asset_value_today_vec / value_total_today

# Find the corresponding geogr. allocation in %
alloc_weights_today_vec = exposure_matrix @ asset_weights_today_vec

# Geogr. value allocation given today's allocation
alloc_value_today_vec = value_total_today * alloc_weights_today_vec

# Total Portfolio value after investment
value_total_new = value_total_today + invest

# Geogr. value allocation given the target allocation
alloc_value_new_vec = value_total_new * alloc_target_vec

alloc_value_diff_vec = alloc_value_new_vec - alloc_value_today_vec

print(f"Total Value Today : {int(value_total_today.round(0))} CHF")
print(f"Asset Allocation [CHF] : {asset_value_today_vec.round(0).astype(int)}")
print(f"Asset Allocation [%] : {asset_weights_today_vec.round(4)*100}")
print(f"Geogr. Allocation [%] : {alloc_weights_today_vec.round(4)*100}")
print(f"Total Value New : {int(value_total_new.round(0))} CHF")
print(f"Geogr. Allocation [CHF] : {alloc_value_new_vec.round(0).astype(int)}")
print(f"Geogr. Allocation, Difference [CHF] : {alloc_value_diff_vec.round(0).astype(int)}")

Total Value Today : 70611 CHF
Asset Allocation [CHF] : [29089 11624  6586  5526  1978   283 15524]
Asset Allocation [%] : [41.2  16.46  9.33  7.83  2.8   0.4  21.99]
Geogr. Allocation [%] : [11.05 24.84 42.79  9.86 11.46]
Total Value New : 160611 CHF
Geogr. Allocation [CHF] : [24092 32122 72275 24092  8031]
Geogr. Allocation, Difference [CHF] : [16291 14581 42057 17133   -62]


In [3]:
exposure_matrix_var = asset_data.loc[asset_data['Is_Variable'] == 1, columns_to_keep].to_numpy(dtype=float).T
print(exposure_matrix_var)
x = np.linalg.inv(exposure_matrix_var.T @ exposure_matrix_var) @ exposure_matrix_var.T @ alloc_value_diff_vec
print(x)
y = exposure_matrix_var @ x
print(y)
z = y + alloc_value_today_vec
print(z)
print(alloc_value_new_vec)

[[0.0198 0.143  1.    ]
 [0.1214 0.8512 0.    ]
 [0.6338 0.     0.    ]
 [0.1762 0.     0.    ]
 [0.0488 0.0058 0.    ]]
[68192.24714153  7376.61586079 13886.36005301]
[16291.42261451 14557.51422369 43220.2462383  12015.47394634
  3370.5660325 ]
[24091.59891777 32098.73937469 73437.55633039 18974.40157818
 11463.58630608]
[24091.59891777 32122.13189036 72274.7967533  24091.59891777
  8030.53297259]


In [4]:
# ==========================================
# Constrained Optimization (The Realistic 'x')
# ==========================================

from scipy.optimize import minimize

# 1. Define the objective: Minimize the distance between our new exposure and the target difference
def objective(x, M, z):
    return np.sum((M @ x - z)**2)

# 2. Get the number of variable assets to set up our parameters
num_var_assets = exposure_matrix_var.shape[1]

# 3. Initial guess: Split the 90,000 CHF evenly to give the solver a starting point
x0 = np.ones(num_var_assets) * (invest / num_var_assets)

# 4. Bounds: No short selling (0 CHF minimum, no maximum)
bounds = [(0, None)] * num_var_assets

# 5. Constraint: The sum of our trades MUST exactly equal the 'invest' budget
constraints = {'type': 'eq', 'fun': lambda x: np.sum(x) - invest}

# 6. Run the SLSQP solver
result = minimize(
    objective, 
    x0, 
    args=(exposure_matrix_var, alloc_value_diff_vec), 
    method='SLSQP', 
    bounds=bounds, 
    constraints=constraints
)

# Extract your optimal, budget-constrained asset purchases in CHF!
x = result.x

print(f"Optimal Variable Asset Purchases [CHF]: {x.round(0).astype(int)}")

Optimal Variable Asset Purchases [CHF]: [68482  7509 14009]
